we're using the navier stokes equation here however without explicitly simulating pressure. turns out simulating pressure is especially difficult due to... reasons? i think we dont have an easy equation to calculate pressure like we do for other things and it's easier to calculate what pressure should be rather than what it is which is interesting.

so the general idea is using the navier stokes equation but instead of calculating pressure, solve the navier stokes equation ignoring pressure. the result of that equation tells us what our wind movement in x,y looks like. we can then use that info to solve the poisson equation. from my understanding the poisson equations finds us pressure values that would make sense in the context of weather. so for example if theres a lot of wind moving to the right it will assign the right a high pressure value and the surrounding areas lower pressure.

with these "solved for" pressure values, we actually update the x,y wind movement with the pressure values to get a realistic? simulation of wind

In [1]:
import numpy as np

In [105]:
timestep = 1
cellSize = 1

viscosity = 10**-5
density = 1.2

In [8]:
xWindMovement = np.arange(1, 26).reshape(5, 5)
yWindMovement = np.arange(1, 26).reshape(5, 5)

In [99]:
def calculateNeighbors(data):
    easternNeighbor = np.hstack((data[:, 1:], np.zeros((5,1))))
    westernNeighbor = np.hstack((np.zeros((5,1)), data[:, :-1]))
    northernNeighbor = np.vstack((np.zeros(5), data[:-1]))
    southernNeighbor = np.vstack((data[1:], np.zeros(5)))

    return (northernNeighbor, easternNeighbor, southernNeighbor, westernNeighbor)

def divergence(xVelocity, yVelocity):
    _,e,_,w = calculateNeighbors(xVelocity)
    n,_,s,_ = calculateNeighbors(yVelocity)
    return (e-w)/cellSize*2 + (n-s)/cellSize*2

In [72]:
divergence(xWindMovement, yWindMovement)

array([[ -2. ,  -2.5,  -3. ,  -3.5,  -7. ],
       [ -1.5,  -4. ,  -4. ,  -4. ,  -9.5],
       [  1. ,  -4. ,  -4. ,  -4. , -12. ],
       [  3.5,  -4. ,  -4. ,  -4. , -14.5],
       [ 19. ,   9.5,  10. ,  10.5,  -2. ]])

In [96]:
prevPressure = np.full((5,5), np.inf)
pressure = np.zeros((5, 5))
stabalizedThresold = 0.001
numIters = 0
while np.abs(pressure - prevPressure).max() > stabalizedThresold:
    summedNeighborPressure = np.add.reduce(np.array(calculateNeighbors(pressure)))
    cellDivergence = divergence(xWindMovement, yWindMovement)
    currentPressure = (summedNeighborPressure - cellDivergence)/4

    prevPressure = pressure
    pressure = currentPressure
    numIters += 1

print(f'After {numIters} iterations...')
print(pressure)

After 49 iterations...
[[ 2.13952055  3.87335972  5.02643813  5.54861222  5.15871245]
 [ 2.68555865  5.82881807  7.68545293  8.51063621  8.08707376]
 [ 1.27523383  5.07340947  7.37859504  8.72391447  9.18028429]
 [-1.65636051  1.8136666   4.03494793  5.82881807  7.91182126]
 [-6.21300467 -4.19482205 -2.87861233 -1.35290289  2.13952055]]


In [101]:
n,e,s,w = calculateNeighbors(xWindMovement)
xPredictedWind = xWindMovement * (e - w)/(2*cellSize) - yWindMovement * (n - s)/(2*cellSize)
xDiffusion = viscosity * ((e - 2*xWindMovement + w)/(cellSize**2) + (n - 2*xWindMovement + s)/(cellSize**2))

n,e,s,w = calculateNeighbors(yWindMovement)
yPredictedWind = xWindMovement * (e - w)/(2*cellSize) - yWindMovement * (n - s)/(2*cellSize)
yDiffusion = viscosity * ((e - 2*yWindMovement + w)/(cellSize**2) + (n - 2*yWindMovement + s)/(cellSize**2))

In [106]:
n,e,s,w = calculateNeighbors(pressure)

xPressureGradient = (e-w)/(2*cellSize)
yPressureGradient = (n-s)/(2*cellSize)

# we multiply by timestep to scale the pressure according to timestep - more timesteps passing = greater movement as a result
# we divide by density to scale the pressure according to how difficult it is to move air. for a very dense substance, it takes greater force, greater pressure to move it compared to low density fluid which will move easily
xWindMovement = xWindMovement - timestep/density * xPressureGradient
yWindMovement = yWindMovement - timestep/density * yPressureGradient